# **Second assignment: Decision trees** #

**Assigment done by Marta Longo, Lucas Oliveira, Sara Táboas for Artificial Inteligence Class**

# Table of contents


<!-- Hello World -->
1. [Introduction](#Introduction)
2. [Libraries Needed](#Libraries)
3. [ID3 Algorithm](#ID3-Algorithm)
4. [Entropy and Information Gain](#Entropy-and-Information-Gain)
5. [Train Data](#Train-Data)
6. [Predict Function](#Predict-Function)
7. [Datasets](#Datasets)
8. [Results on Datasets](#Results-on-Datasets)
    - [1. Iris](#Iris)
    - [2. Restaurant](#Restaurant)
    - [3. Weather](#Weather)
    - [4. Connect-4](#Connect-4)
9. [Menu](#Menu)
10. [Conclusion](#Conclusion)

# Introduction 

The purpose of this assigment is to write a program that learns a decision tree from three training datasets that were given to us using the ID3 algorithm.

# Libraries
[[ go back to the top ]](#Table-of-contents)

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np
import matplotlib.pyplot as plt
import math

# ID3 Algorithm
[[ go back to the top ]](#Table-of-contents)

The ID3 algorithm is a popular method used to create decision trees, which are a type of predictive model used in machine learning and statistics. ID3 works by selecting the attribute that provides the highest information gain to split the data, aiming to reduce entropy (impurity) in the dataset. The process is repeated recursively for each subset, with the attribute that best separates the data chosen at each step, forming a tree structure where each node represents a decision based on an attribute. The recursion stops when all instances in a subset belong to the same class, no more attributes are available, or there are no more instances to split. Here are the key elements:

## **Implementation**<a id="Implementation"></a>

In [ ]:
class TreeNode:
    def __init__(self, feature_index=None, value=None, label=None, counter=0, children=None, feature_name = None):
        self.feature_index = feature_index  # Index of the feature used for splitting
        self.value = value  # Value used for splitting (if categorical)
        self.label = label  # Class label (if leaf node)
        self.counter = counter  # Counter for the number of examples
        self.children = children  # Dictionary of children nodes
        self.feature_name = feature_name  #the feature that was selected
      
    
    def __str__(self):
        return ("Index: " + str(self.feature_index) + "\nValue: " + str(self.value) + "\nLabel:" + str(self.label) + "\nChildren:" + str(self.children))
    
 
    def print_tree(self, indent="", instance_counter=None):
        if instance_counter is None:
            instance_counter = [0]  # Initialize the counter as a list to pass by reference

        instance_counter[0] += 1  # Increment the counter for each node visited
        if self.label is not None:
            return f"{self.label} ({instance_counter[0]})"
        
        result = []
        if isinstance(self.value, list):  # Handling categorical split
            result.append(f"\n{indent}    <{self.feature_name}>")
            for v in self.children:
                child = self.children[v]
                result.append(f"{indent}        {v}: {child.print_tree(indent + '        ', instance_counter)}")
        else:  # Handling numerical split
            result.append(f"\n{indent}    <{self.feature_name}>")
            result.append(f"{indent}        <= {self.value}: {self.children['left'].print_tree(indent + '        ', instance_counter)}")
            result.append(f"{indent}        > {self.value}: {self.children['right'].print_tree(indent + '        ', instance_counter)}")
        
        return "\n".join(result)
    

class DecisionTree:
    def __init__(self, max_depth=None, min_samples_split=2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.root = None

    def entropy(self, labels):
        # Calculate entropy (measure of impurity)
        from collections import Counter
        counts = Counter(labels)
        probabilities = [count / len(labels) for count in counts.values()]
        entropy = 0
        for p in probabilities:
            log_value = math.log2(p)
            entropy += -p * log_value
        return entropy

    def cat_information_gain(self, data, feature_index, values):
        labels = {}
        for v in values:
            labels[v] = []
        for row in data:
            for v in values:
                if row[feature_index] == v:
                    labels[v].append(row[-1])
        result = self.entropy([label for row in data for label in row[-1]])
        for v in values:
            
            result -= self.entropy(labels[v]) * len(labels[v])/len(data)
        return result

    def information_gain(self, data, feature_index, threshold):
        # Calculate information gain for a split
        left_labels, right_labels = [], []
        for row in data:
            if row[feature_index] <= threshold:
                left_labels.append(row[-1])
            else:
                right_labels.append(row[-1])
        parent_entropy = self.entropy([label for row in data for label in row[-1]])
        left_entropy = self.entropy(left_labels)
        right_entropy = self.entropy(right_labels)
        weight = len(left_labels) / len(data)
        return parent_entropy - (weight * left_entropy + (1 - weight) * right_entropy)

    def fit(self, data, feature_name):
        # Train the decision tree
        self.feature_name = feature_name
        self.root = self._build_tree(data, 0)
    
    def _build_tree(self, data, depth):
        # Recursive function to build the tree
        if depth >= self.max_depth or len(data) < self.min_samples_split or len(set([row[-1] for row in data])) == 1:
        # Stopping conditions: reach max depth, min samples, or pure data
            aux = majority_vote(data)
            return TreeNode(label=aux)  # Assign majority class label

        best_gain, best_feature_index, best_threshold = 0, None, None
        evaluated_data_type = None
        best_values = []
        for feature_index in range(len(data[0]) - 1):
            if type(data[1][feature_index]) != str:
                # Find the best split among all features
                values = set([row[feature_index] for row in data])
                for value in values:
                    gain = self.information_gain(data.copy(), feature_index, value)
                    if gain > best_gain:
                        best_values = values.copy()
                        best_gain, best_feature_index, best_threshold = gain, feature_index, value
                        evaluated_data_type = "num"         
            else:
                values = [item for item in set([row[feature_index] for row in data])]
                gain = self.cat_information_gain(data.copy(), feature_index, values)
                if gain > best_gain:
                    best_values = values.copy()
                    best_gain, best_feature_index, best_threshold = gain, feature_index, 0
                    evaluated_data_type = "cat"
        

        sets_of_data = {}
        children = {}
        if evaluated_data_type == "cat":
            for v in best_values:
                sets_of_data[v] = []
            sets_of_data["default"] = []
            for row in data:
                for v in best_values:
                    if row[best_feature_index] == v:
                        sets_of_data[v].append(row)
            for v in best_values:
                children[v] = self._build_tree(sets_of_data[v], depth + 1)
            children["default"] = TreeNode(label=majority_vote(data))
                
            return TreeNode(
                feature_index=best_feature_index,
                value=best_values,
                children=children,
                feature_name=self.feature_name[0][best_feature_index]
                
            )
        else:
            sets_of_data["left"] = []
            sets_of_data["right"] = []
            for row in data:
                if row[best_feature_index] <= best_threshold:
                    sets_of_data["left"].append(row)
                else:
                    sets_of_data["right"].append(row)
            children["left"] = self._build_tree(sets_of_data["left"], depth + 1)
            children["right"] = self._build_tree(sets_of_data["right"], depth + 1)
            return TreeNode(
                feature_index=best_feature_index,
                value=best_threshold,
                children=children,
                feature_name=self.feature_name[0][best_feature_index]
            )

    def predict(self, datapoint):
        # Predict class label for a new data point
        node = self.root
        counter = 0
        while node.label is None:
            if type(node.value) is list:
                if not node.value.__contains__(datapoint[node.feature_index]):
                    node = node.children["default"]
                    counter += 1
                    continue
                for value in node.value:
                    if datapoint[node.feature_index] == value:
                        node = node.children[str(value)]
                        counter += 1
                        break
            else:
                if datapoint[node.feature_index] <= node.value:
                    node = node.children["left"]
                    counter += 1
                else:
                    node = node.children["right"]
                    counter += 1
        return node.label
    

    def print_tree(self):
        if self.root:
            return self.root.print_tree()
        
        return "Tree is empty."



# Helper function to find the majority class label
def majority_vote(data):
  labels = [row[-1] for row in data]
  from collections import Counter
  counts = Counter(labels)
  return counts.most_common(1)[0][0]

**TreeNode Class Attributes**
- **feature_index**: This is the index of the feature used to split the data at this node. It identifies which feature (among the input features) is being used to decide the split at this node. If this node is a leaf node (i.e., it does not split further), feature_index will be None.


- **value**: This is the value used for splitting the data at this node if the feature is continuous. For categorical features, this could be the category value. It represents the threshold or category against which the feature values are compared. If this node is a leaf node, value will be None.

- **label**: This is the class label that the node predicts if it is a leaf node. For non-leaf nodes, this will be None.

- **counter**: This represents the count of examples that reach this node during the tree building process.

- **children**: This is a dictionary that stores the children nodes of the current node. Each key in the dictionary represents a particular value or range of values of the feature associated with the current node. The values associated with these keys are the child nodes resulting from splitting the data based on the corresponding feature value. For categorical features, the keys represent distinct categories, while for continuous features, the keys can represent different partitions of the feature space.

- **feature_name**: This is the name/label of the feature associated with the current node.

**Decision Tree Class Attributes**
- **max_depth**: This specifies the maximum depth that the tree is allowed to grow to. If None, the tree will grow until all leaves are pure or until there are fewer than min_samples_split samples in the leaves. Limiting the depth helps in controlling overfitting.

- **min_samples_split**: This defines the minimum number of samples required to split an internal node. This parameter prevents the tree from splitting too much and helps in controlling overfitting. If the number of samples at a node is less than min_samples_split, the node will not be split further.

- **root**: This is the root node of the decision tree. It is the starting point for making predictions. Initially, this is None until the tree is fitted with data.

## **Entropy and Information Gain**

[[ go back to the top ]](#Table-of-contents)

In [ ]:
'''
def entropy(self, labels):
        # Calculate entropy (measure of impurity)
        from collections import Counter
        counts = Counter(labels)
        probabilities = [count / len(labels) for count in counts.values()]
        print("prob = ", probabilities) 
        # from math import log2
        entropy = 0
        for p in probabilities:
            log_value = math.log2(p)
            entropy += -p * log_value
        return entropy


def cat_information_gain(self, data, feature_index, values):
    labels = {}
    for v in values:
        labels[v] = []
    for row in data:
        for v in values:
            if row[feature_index] == v:
                labels[v].append(row[-1])
    result = self.entropy([label for row in data for label in row[-1]])
    for v in values:
        
        result -= self.entropy(labels[v]) * len(labels[v])/len(data)
    return result

    
def information_gain(self, data, feature_index, threshold):
    # Calculate information gain for a split
    left_labels, right_labels = [], []
    for row in data:           
        if row[feature_index] <= threshold:
            left_labels.append(row[-1])
        else:
            right_labels.append(row[-1])
    parent_entropy = self.entropy([label for row in data for label in row[-1]])
    left_entropy = self.entropy(left_labels)
    right_entropy = self.entropy(right_labels)
    weight = len(left_labels) / len(data)
    return parent_entropy - (weight * left_entropy + (1 - weight) * right_entropy)
'''

### 1. **entropy(self, labels):** To calculate the entropy of a set of labels. Entropy is a measure of the impurity or randomness in the dataset.

#### Functionality
- **Count occurrences**: Uses Counter from the collections module to count the occurrences of each unique label.
- **Calculate probabilities**: Computes the probability of each label by dividing the count of each label by the total number of labels.
- **Calculate entropy**: Uses the formula $$\text{Entropy} = -\sum(p \log_2 p)$$  where \( p \) is the probability of a label. This is summed over all unique labels.
- **Return entropy**: Returns the calculated entropy value.

### **2. cat_information_gain(self, data, feature_index, values):** To calculate the information gain for a categorical feature. Information gain measures the reduction in entropy achieved by splitting the data based on a feature.
- **Initialize labels dictionary**: Creates a dictionary labels to hold lists of class labels for each unique value of the categorical feature.
- **Distribute labels**: Iterates over the rows in the dataset and assigns the class label (last element in the row) to the appropriate list in the labels dictionary based on the feature value.
- **Calculate parent entropy**: Computes the entropy of the entire dataset's class labels using the entropy method.
- **Calculate information gain**:
  - For each unique value in the feature, calculate the entropy of the corresponding subset of labels.
  - Adjust the parent entropy by subtracting the weighted entropy of each subset (proportional to its size).
- **Return information gain**: Returns the resulting information gain.

### **3. information_gain(self, data, feature_index, threshold):** To calculate the information gain for a numerical feature based on a given threshold. This is used to evaluate the best threshold for splitting the data.

- **Initialize label lists**: Creates two lists, left_labels and right_labels, to hold class labels for the left and right subsets created by the threshold split.
- **Distribute labels**: Iterates over the rows in the dataset and assigns the class label (last element in the row) to either left_labels or right_labels based on whether the feature value is less than or equal to the threshold.
- **Calculate parent entropy**: Computes the entropy of the entire dataset's class labels using the entropy` method.
- **Calculate subset entropies**: Computes the entropy of left_labels and right_labels.
- **Calculate weighted average entropy**: Computes the weighted average of the left and right entropies based on their sizes relative to the total dataset.
- **Return information gain**: Subtracts the weighted average entropy from the parent entropy to get the information gain.

### **Summary**

- **entropy**: Calculates the entropy of a list of labels, providing a measure of impurity.
- **cat_information_gain**: Calculates the information gain for a categorical feature, assessing how much splitting the data on this feature reduces impurity.
- **information_gain**: Calculates the information gain for a numerical feature using a specific threshold, evaluating the effectiveness of this split in reducing impurity.

These functions are essential for decision tree algorithms, helping to determine the best features and thresholds for splitting the data to achieve the most significant reduction in impurity.

## **Train Data**

[[ go back to the top ]](#Table-of-contents)

In [ ]:
'''
def fit(self, data):
        # Train the decision tree
        self.root = self._build_tree(data, 0)
    
def _build_tree(self, data, depth):
    # Recursive function to build the tree
    if depth >= self.max_depth or len(data) < self.min_samples_split or len(set([row[-1] for row in data])) == 1:
    # Stopping conditions: reach max depth, min samples, or pure data
        aux = majority_vote(data)
        return TreeNode(label=aux)  # Assign majority class label

    best_gain, best_feature_index, best_threshold = 0, None, None
    evaluated_data_type = None
    best_values = []
    for feature_index in range(len(data[0]) - 1):
        if type(data[1][feature_index]) != str:
            # Find the best split among all features
            values = set([row[feature_index] for row in data])
            for value in values:
                gain = self.information_gain(data.copy(), feature_index, value)
                if gain > best_gain:
                    best_values = values.copy()
                    best_gain, best_feature_index, best_threshold = gain, feature_index, value
                    evaluated_data_type = "num"
                   
        else:
            
            values = [item for item in set([row[feature_index] for row in data])]
            gain = self.cat_information_gain(data.copy(), feature_index, values)
            if gain > best_gain:
                best_values = values.copy()
                best_gain, best_feature_index, best_threshold = gain, feature_index, 0
                evaluated_data_type = "cat"
               
  
    sets_of_data = {}
    children = {}
    if evaluated_data_type == "cat":
        for v in best_values:
            sets_of_data[v] = []
        for row in data:
            for v in best_values:
                if row[best_feature_index] == v:
                    sets_of_data[v].append(row)
        for v in best_values:
            children[v] = self._build_tree(sets_of_data[v], depth + 1)
        return TreeNode(
            feature_index=best_feature_index,
            value=best_values,
            children=children
        )
    else:
        sets_of_data["left"] = []
        sets_of_data["right"] = []
        for row in data:
            if row[best_feature_index] <= best_threshold:
                sets_of_data["left"].append(row)
            else:
                sets_of_data["right"].append(row)
        children["left"] = self._build_tree(sets_of_data["left"], depth + 1)
        children["right"] = self._build_tree(sets_of_data["right"], depth + 1)
        return TreeNode(
            feature_index=best_feature_index,
            value=best_threshold,
            children=children
        )
       
    # Helper function to find the majority class label
def majority_vote(data):
  labels = [row[-1] for row in data]
  from collections import Counter
  counts = Counter(labels)
  return counts.most_common(1)[0][0]
'''

### **1. fit(self, data, feature_name):** To train the decision tree by building the tree structure recursively using the provided dataset.
- **self.feature_name = feature_name**: We indicate what features are there to later use them on the print function
- **self.root = self._build_tree(data, 0)**: Calls the _build_tree method to construct the decision tree starting from the root node.

### **2. _build_tree(self, data, depth):** To recursively build the decision tree.
- **Stopping Conditions**:
  - Checks if any of the stopping conditions are met:
    - If the depth of the tree exceeds the maximum depth specified (self.max_depth).
    - If the number of samples in the node is less than the minimum required for a split (self.min_samples_split).
    - If all samples belong to the same class, indicating pure data.
  - If any stopping condition is met, it returns a leaf node with the majority class label.
- **Finding the Best Split**:
  - Iterates over each feature to find the best split:
    - For numerical features:
      - Finds the best threshold to split the data based on information gain.
    - For categorical features:
      - Finds the best feature and its values to split the data based on categorical information gain.
  - It tracks the feature index, threshold (for numerical features), and information gain for the best split.
- **Splitting the Data**:
  - Divides the dataset into subsets based on the best split:
    - For categorical features: Creates subsets for each unique value of the best feature.
    - For numerical features: Creates subsets based on whether the feature value is less than or equal to the threshold.
- **Recursively Building Children Nodes**:
  - Recursively calls _build_tree on each subset to build children nodes.
- **Creating the TreeNode**:
  - Constructs a TreeNode object representing the current node in the decision tree.
  - Sets attributes such as feature index, splitting value/threshold, children nodes and feature name.

#### **Helper Function**
- **majority_vote(data)**:
  - Finds the majority class label in a dataset.
  - It counts the occurrences of each class label and returns the most common class label.

### **Summary**
- **fit method**: Trains the decision tree model by building the tree structure.
- **_build_tree method**: Recursively constructs the decision tree by finding the best splits and creating nodes.
- **majority_vote function**: Finds the majority class label in a dataset, which is used for leaf nodes.

## **Predict Function** 

[[ go back to the top ]](#Table-of-contents)

In [ ]:
'''
def predict(self, datapoint):
    # Predict class label for a new data point
    node = self.root
    counter = 0
    while node.label is None:
        if type(node.value) is list:#CAT
            for value in node.value:
                if datapoint[node.feature_index] == value:
                    node = node.children[str(value)]
                    counter += 1
                    break
        else:
            if datapoint[node.feature_index] <= node.value:
                node = node.children["left"]
                counter += 1
            else:
                node = node.children["right"]
                counter += 1
    return node.label
'''

### **1. predict(self, datapoint):** To predict the class label for a given data point by navigating through the decision tree structure.

- **Initialization**:
  - **node = self.root**: Starts at the root of the decision tree.
  - **counter = 0**: Initializes a counter to track the number of nodes visited (though it isn't used beyond incrementing).

- **Tree Traversal**:
  - **while node.label is None:**: Continues to traverse the tree until a leaf node (which has a label) is reached.
    - **Categorical Features**:
      - **if type(node.value) is list:**: Checks if the current node's value is a list, indicating a categorical feature.
        - **Iterate over possible values**: Loops through the possible values of the categorical feature.
        - **Match value and move to child node**: If the feature value in the data point matches the current value, it moves to the corresponding child node and breaks out of the loop.
    - **Numerical Features**:
      - **else:**: Handles numerical features where the node's value is not a list.
        - **Compare and move left or right**:
          - **if datapoint[node.feature_index] <= node.value:**: If the feature value in the data point is less than or equal to the node's value, it moves to the left child.
          - **else:**: Otherwise, it moves to the right child.

- **Return the Class Label**:
  - **return node.label**: Once a leaf node is reached, the method returns the class label associated with that leaf node.

### **Summary**
- **Initialization**: Starts from the root node of the decision tree.
- **Traversal**: Navigates through the tree based on the feature values of the data point. It differentiates between categorical and numerical features to decide which child node to move to.
- **Termination**: Stops when a leaf node is reached and returns the class label of that leaf node.

### **Example**
1. **Starting at the Root Node**: The function begins at the root node.
2. **Checking Feature Type**: Determines if the feature is categorical or numerical.
3. **Traversing the Tree**: Moves to the appropriate child node based on the feature value in the data point.
4. **Repeating Steps**: Continues the process until it reaches a leaf node.
5. **Returning the Label**: Once at a leaf node, it returns the class label stored in that node.

This method ensures that the class label is predicted accurately by following the structure and decisions defined by the trained decision tree.

## **Datasets**

[[ go back to the top ]](#Table-of-contents)

We created a class Data to analyse the type of data and do the split in test and train data. 

In [ ]:
class Data:
    def __init__(self, data, split, name=None):
        
        print(data)                                                 # visualize the dataset
        data = data.fillna("Empty")                                 # replace the NAN values with a string so it was a type                                         
        self.name = name                                            # dataset name
        self.classes = np.unique(data[list(data)[-1]])              # list of classes
        self.split = split                                          # split rate between training and test sets
        self.train, self.test = self._splitSets(data, self.split)   # split datasets into training and test sets
        self.columns = self._getColumns(data)                       # list of column names
        self.tree = DecisionTree(max_depth=5)                       # decision tree
        self.tree.fit(self.train, self.columns)                     # trains the decision tree
        
    def _getColumns(self, data):
        names = [str(column) for column in data.columns]
        types = [type(column) for column in data.iloc[1]]
        names.pop()
        names.pop(0)
        types.pop()
        types.pop(0)
        return (names, types)
    
    def testPredict(self):
        sum = 0
        for row in self.test:
            if self.tree.predict(row) == row[-1]:
                sum += 1

        return sum/len(self.test)
        
    def _splitSets(self, data, split):                              #split dataset between training and test sets with the same proportion of rows from each class
        train = None
        test = None
        if data.columns.__contains__("ID"):                         # remove ID column if it exists
            data = data.drop("ID", axis=1)
        dataOfClass = self._splitByClass(data)                      # split dataset by class
        for cl in dataOfClass:                                      # split dataset in two keeping the proportion of rows from each class
            currentData = dataOfClass[cl]
            if len(currentData) == 0:
                continue
            sampleSize = int(split * len(currentData))
            if train is None:
                test = currentData.sample(sampleSize, replace=False)
                train = currentData.drop(test.index)
            else:
                tempTest = currentData.sample(sampleSize, replace=False)
                tempTrain = currentData.drop(tempTest.index)
                train = pd.concat([train, tempTrain], axis=0)
                test = pd.concat([test, tempTest], axis=0)
        return train.values.tolist(), test.values.tolist()          # return training and test sets as lists
        
    def _splitByClass(self, data):                                  # split the dataset in different datasets containing only rows from the same class
        classColumn = list(data)[-1]
        classes = np.unique(data[classColumn])
        dataOfClass = {}
        for class_label in classes:
            dataOfClass[class_label] = data[data[classColumn] == class_label]   # remove the class column from the dataframe
        return dataOfClass                                          # returns a dictionary {"class" : dataframeContainingOnlyRowsOfClass}
    
    def _transform(self, data):                                     # transform all the columns, turning categorical data into numerical data, except for the class column
        enc = LabelEncoder()
        new_data = data.copy()
        for i, col in enumerate(data.columns):
            if i == len(data.columns)-1:
                return
            if (type(data[col][0]) != str):
                continue
            new_data[col] = enc.fit_transform(new_data[col])
        return new_data

### **1. __init__(self, data, split, name):** The constructor initializes the Data object with a dataset and a split ratio for training and test sets. It also performs some initial preprocessing.
- data = data.fillna("Empty"): This replaces any missing values in the dataset with the string "Empty".
- name: Dataset name
- print(data): Prints the dataset to the console for visualization.
- self.classes = np.unique(data[list(data)[-1]]): Extracts the unique classes from the last column of the dataset.
- self.split = split: Sets the split ratio for training and test sets.
- self.train, self.test = self._splitSets(data, self.split): Splits the dataset into training and test sets using the _splitSets method.

### **2. _getColumns(self, data):** To get column names and their respective type
- Removes the first and last columns 
- Returns: A tuple containing lists of column names and their corresponding types

### **3. _testPredict(self):** To evaluate the performance of the trained decision tree on our test set
- It iterates through each row in the test set, predicts the class label using the decision tree, and compares it with the actual class label.
- Returns: the accuracy of the decision tree on the test set.

### **4. _splitSets(self, data, split):** To split the dataset into training and test sets while maintaining the same proportion of rows from each class.
- Removes ID column: If the dataset contains an "ID" column, it is removed.
- Split by class: Calls _splitByClass to get a dictionary of dataframes, each containing only rows of a single class.
- Proportional splitting:
    For each class-specific dataframe:
        - Samples a portion of rows for the test set based on the split ratio.
        - The remaining rows go into the training set.
- Concatenates these class-specific splits into the overall training and test sets.
- Returns: The training and test sets as lists of rows.

### **5. _splitByClass(self, data):** To split the dataset into separate dataframes, each containing only rows of a single class.
- Identify the class column: Assumes the last column of the dataset contains the class labels.
- Create class-specific dataframes:
    - Iterates through each unique class label.
    - Filters the original dataset to create a dataframe containing only rows of the current class.
- Returns: A dictionary where keys are class labels and values are dataframes with only rows of the corresponding class.

### **6. _transform(self, data):** To transform categorical data into numerical data, except for the class column.
- Label encoding: Initializes a LabelEncoder from scikit-learn.
- Transform each column:
    - Iterates over the columns of the dataset.
    - Skips the last column (assumed to be the class column).
    - Transforms categorical columns (identified as string columns) into numerical data using the label encoder.
- Returns: A new dataframe with transformed columns. If a column is not categorical, it remains unchanged.

### **Summary**

__init__: Initializes the dataset, handles missing values, and splits the data into training and test sets.
**_splitSets:** Splits the dataset into training and test sets while maintaining class proportions.
**_splitByClass:** Creates separate dataframes for each class in the dataset.
**_transform:** Encodes categorical data into numerical values for all columns except the class column.

These functions collectively ensure that the data is preprocessed, split, and ready for machine learning tasks while preserving class distributions.


We were given four datasets to work with.

## **Results on Datasets**

[[ go back to the top ]](#Table-of-contents)

For each dataset, we started by using the entire dataset to train and build our decision tree classifier. Then, when added a new line as input, out classifier is able to predict the most likely class value. 

Additionally, it is possible to use the four datasets available to do a train/test split by choosing the percentage of data of each set. We defined a standard train/test split of 70%/30%.

## **Iris Analysis**
[[ go back to the top ]](#Table-of-contents)

Iris Dataset contains numerical information about plants of three classes: iris setosa, iris virginica
and iris versicolor. The attributes are petal length and width and sepal length and width. The
task is to learn a decision tree that can tell to which class a plant belongs to, given its sepal and
petal lengths and widths.

In [ ]:
data = pd.read_csv('datasets/iris.csv')
Iris = Data(data, 0.3, "Iris") #creates an instance of class Data and we indicate the split is 0.3 for test data 

Here we can see that most of the instances in 'iris.csv' are floats and then there are three labels: 'Iris-virginica', 'Iris-setosa' and 'Iris-versicolor'.

In [ ]:
count_classes = pd.value_counts(data['class'], sort = True).sort_index()
count_classes.plot(kind = 'bar')
plt.title("Iris histogram")
plt.xlabel("Class")
plt.ylabel("Frequency")

The classes are equally distributed so there are no class imbalance problems, which indicates that is a good dataset. Firstly, we initialize the Tree:

In [ ]:
dTree = DecisionTree(max_depth=5)

Then we train it:

In [ ]:
dTree.fit(Iris.train, Iris.columns)              #training data is an attribute of Data

After that, we predict:

In [ ]:
final_results = []
sum = 0
for row in Iris.test:
    if dTree.predict(row) == row[-1]: # everytime the prediction is equal to the test in each row of the dataset we increment the counter 
        sum += 1
        
print('Number of right predictions: %d/%d' % (sum,len(Iris.test)))

In [ ]:
from sklearn.metrics import classification_report

# Initialize lists to store true and predicted labels
true_labels = []
predicted_labels = []

# Collect predictions and true labels
for row in Iris.test:
    true_labels.append(row[-1])
    predicted_labels.append(dTree.predict(row))

# Generate the classification report
report = classification_report(true_labels, predicted_labels, target_names=[str(cls) for cls in np.unique(true_labels)])

# Print the classification report
print('Classification Report:')
print(report)

In [ ]:
print(dTree.print_tree())

## **Restaurant**
[[ go back to the top ]](#Table-of-contents)

The Restaurant dataset contains information about customers and
restaurants (type of food, waiting time, price etc), and the class attribute (last column) says if
the customer will wait or not to eat in that restaurant. The task is to generate a decision tree (as
explained in theoretical class and following the ID3 algorithm available in the textbook). This
decision tree must be used later to classify (answer if the customer will wait or not) new cases.


In [ ]:
restaurant_data = pd.read_csv('datasets/restaurant(1).csv')
Restaurant = Data(restaurant_data, 0.3, "Restaurant") 

**0.7**

In [ ]:
count_classes = pd.value_counts(restaurant_data['Class'], sort = True).sort_index()
count_classes.plot(kind = 'bar')
plt.title("Restaurant histogram")
plt.xlabel("Class")
plt.ylabel("Frequency")


In [ ]:
dTree = DecisionTree(max_depth=5)

In [ ]:
dTree.fit(Restaurant.train, Restaurant.columns)              #training data is an attribute of Data

In [ ]:
sum = 0
for row in Restaurant.test:
    print("row_test: ", row)
    if dTree.predict(row) == row[-1]: # everytime the prediction is equal to the test in each row of the dataset we increment the counter 
        sum += 1
        
print('Number of right predictions: %d/%d' % (sum,len(Restaurant.test)))


In [ ]:
print(dTree.print_tree())

## **Weather**
[[ go back to the top ]](#Table-of-contents)

The Weather dataset contains information about climate conditions to play tennis. The task is to learn a
decision tree that can decide what are the best conditions to play tennis.

In [ ]:
weather_data = pd.read_csv('datasets/weather.csv')
Weather = Data(weather_data, 0.3, "Weather") #we create an instance of class Data and we indicate the split is 0.3 for test data 

As we can see most of the attributes are strings (categorical) therefore they use our categorical information gain.

In [ ]:
count_classes = pd.value_counts(weather_data['Play'], sort = True).sort_index()
count_classes.plot(kind = 'bar')
plt.title("Weather histogram")
plt.xlabel("Class")
plt.ylabel("Frequency")

In [ ]:
dTree = DecisionTree(max_depth=5)

In [ ]:
dTree.fit(Weather.train, Weather.columns)              #training data is an attribute of Data

In [ ]:
sum = 0 
for row in Weather.test:
    if dTree.predict(row) == row[-1]: # everytime the prediction is equal to the test in each row of the dataset we increment the counter 
        sum += 1
        
print('Number of right predictions: %d/%d' % (sum,len(Weather.test)))


In [ ]:
print(dTree.print_tree())

## **Connect-4**
[[ go back to the top ]](#Table-of-contents)

This datasets contains all the combinations of the board of Connect-4 game.

In [ ]:
connect4_data = pd.read_csv('datasets/connect4.csv')
connect4_data_headers = [i for i in range(len(connect4_data.columns)-1)]
connect4_data_headers.append("Class")
connect4_data_old_header = [i for i in connect4_data.columns]
connect4_data.columns = connect4_data_headers
connect4_data.loc[len(connect4_data)] = connect4_data_old_header
connect4_data.insert(0, "ID", [i for i in range(len(connect4_data))])
Connect4 = Data(connect4_data, 0.3, "Connect4") 

All the instances here are strings.

In [ ]:
Class = connect4_data.iloc[:, -1]
count_classes = Class.value_counts()
count_classes.plot(kind = 'bar')
plt.title("Connect-4 histogram")
plt.xlabel("Class")
plt.ylabel("Frequency")

In [ ]:
dTree = DecisionTree(max_depth=5)

In [ ]:
dTree.fit(Connect4.train, Connect4.columns)              #training data is an attribute of DatadTree.fit(Weather.train)              #training data is an attribute of Data

In [ ]:
sum = 0 
for row in Connect4.test:
    if dTree.predict(row) == row[-1]: # everytime the prediction is equal to the test in each row of the dataset we increment the counter 
        sum += 1

In [ ]:
print(dTree.print_tree())

In [ ]:
from connect_four import Board 
from tree import Node 
import random

game_board = Board()

def convert_to_response(board):
    resp=""
    row=0
    column=0
    for i in range (42):
        if row == 6:
            row = 0
            column += 1
        if board[5-row][column] == '.':
            board[5-row][column] = 'b'
        elif board[5-row][column] == 'X':
            board[5-row][column] = 'x'
        elif board[5-row][column] == 'O':
            board[5-row][column] = 'o'

        resp = resp + board[5-row][column] + " "
        row+=1
    
    resp = resp.strip()
    return resp

def getAndPredictConnectFour(dataset: Data, resp):
    x = resp.split(" ")
    for i in range(len(x)):
        x[i] = dataset.columns[1][i](x[i])
    return dataset.tree.predict(x)

while not game_board.end:                                            #decision tree plays
    root = Node(game_board, {}, None)
    root.genChildren(game_board.turn)
    pred = {}
    for move in root.children:
        resp = convert_to_response(root.children[move].value.board) # convert the game state to csv-like format
        if getAndPredictConnectFour(Connect4, resp) == 'win':
            pred[move] = getAndPredictConnectFour(Connect4, resp)
    print(pred)
    if not pred:
        column = random.choice(game_board.possibleMoves())
    else:
        column = random.choice(list(pred.keys()))
    game_board.move(column, game_board.turn)
    print(game_board)
    if game_board.winner != None:
        print('The winner is the MACHINE:', game_board.winner)
        break
    if not game_board.end:                                              #human plays
        user = int(input("Insert column:"))
        game_board.move(user, game_board.turn)
        print(game_board)
        if game_board.winner != None:
            print('The winner is HUMAN:', game_board.winner)
            break


## **Menu**
[[ go back to the top ]](#Table-of-contents)

We created a menu were you can choose which tree you want to see, test the decision tree selected and predict unlabeled data with that same three.

In [ ]:
def printTreeOptions():
    print()
    print("Select a Decision Tree:")
    print("1.Iris Tree")
    print("2.Restaurant Tree")
    print("3.Connect4 Tree")
    print("4.Weather Tree")
    print()
    print("0.Get back to menu")
    
def testTree(dataset: Data):
    print(dataset.name, "tree precision:", dataset.testPedict())
    
def getAndPredictInput(dataset: Data):
    toPrint = "Please insert the following data separated by empty spaces "+str(dataset.columns[0])+":\n"
    x = input(toPrint).split(" ")
    for i in range(len(x)):
        x[i] = dataset.columns[1][i](x[i])
    return dataset.tree.predict(x)

menuSelect = 0
secondaryMenuSelect = 0

while True:
    if menuSelect == 0:     # Menu
        secondaryMenuSelect = 0
        print()
        print("Select an Operation:")
        print("1.See Decision Tree")
        print("2.Test Decision Tree")
        print("3.Predict my own data with Tree")
        print("4.Exit")
        menuSelect = int(input())
        if (menuSelect > 4 or menuSelect < 0):
            menuSelect = 0
            continue
        continue
    elif menuSelect == 1:   # Print a Decision Tree
        printTreeOptions()
        secondaryMenuSelect = int(input())
        
        if (secondaryMenuSelect > 4 or secondaryMenuSelect < 0):
            secondaryMenuSelect = 0
            continue
        if secondaryMenuSelect == 0:
            menuSelect = 0
            continue
        
        if secondaryMenuSelect == 1:    # Iris
            print(Iris.tree.print_tree())
        elif secondaryMenuSelect == 2:  # Restaurant
            print(Restaurant.tree.print_tree())
        elif secondaryMenuSelect == 3:  # Connect4
            print(Connect4.tree.print_tree())
        else:                           # Weather
            print(Weather.tree.print_tree())

    elif menuSelect == 2:   # Test a Decision Tree using the testing data from the csv file

        printTreeOptions()
        secondaryMenuSelect = int(input())
        
        if (secondaryMenuSelect > 4 or secondaryMenuSelect < 0):
            secondaryMenuSelect = 0
            continue
        if secondaryMenuSelect == 0:
            menuSelect = 0
            continue

        if secondaryMenuSelect == 1:    # Iris
            testTree(Iris)
        elif secondaryMenuSelect == 2:  # Restaurant
            testTree(Restaurant)
        elif secondaryMenuSelect == 3:  # Connect4
            testTree(Connect4)
        else:                           # Weather
            testTree(Weather)

    elif menuSelect == 3:   # Insert data and get a prediction from one of the Decision Trees
        printTreeOptions()
        secondaryMenuSelect = int(input())
        
        if (secondaryMenuSelect > 4 or secondaryMenuSelect < 0):
            secondaryMenuSelect = 0
            continue
        if secondaryMenuSelect == 0:
            menuSelect = 0
            continue
        
        if secondaryMenuSelect == 1:    # Iris
            print(getAndPredictInput(Iris))
        elif secondaryMenuSelect == 2:  # Restaurant
            print(getAndPredictInput(Restaurant))
        elif secondaryMenuSelect == 3:  # Connect4
            print(getAndPredictInput(Connect4))
        else:                           # Weather
            print(getAndPredictInput(Weather))

    elif menuSelect == 4:   # Exit the program
        break

# **Conclusion**
[[ go back to the top ]](#Table-of-contents)

We created a Induced ID3 Decision Tree Algorithm that can classify both categorical and numerical values and predict a unlabeled instance of the dataset. We also made a class Data to pre-process all the data and do the intended splits. We obtained very good results, since all our right previsions are at least above 65%. 